<a href="https://colab.research.google.com/github/secutron/Practice_Ignite/blob/main/2_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##2.1.구글 Colab 기반 

이그나이트는 작성 시점을 기준으로 다음의 백엔드들을 지원한다.

- backends from native torch distributed configuration: “nccl”, “gloo”, “mpi”
- XLA on TPUs via pytorch/xla
- using Horovod framework as a backend

각 의미를 이해하기 위해 가능한 단순한 내용의 코드를 준비하고, 이를 구글 Colab 기반으로 동작시켜 본다.


### 2.1.1. CPU단 분산처리

  구글 Colab은, 별도로 런타임 유형을 지정하지 않는 경우 GPU나 TPU가 없는 VM(Virtual Machine)이 기본 할당된다. 이는 Colab 페이지의 상단메뉴에서 ‘런타임’ > ‘런타임 유형 변경’ 선택 시 나오는 대화상자가, 아래 그림에서처럼 하드웨어 가속기가 None으로 표시되는 상태인 것으로 확인 가능하다.

 이 상태에서 기본 CPU만을 이용하여 분산처리를 진행해보자.  

<div align="center">
<img width=512 src="https://i.imgur.com/WLjhi9E.png"/>
</div>



####2.1.1.1. 패키지 설치

 먼저 현 시점 기준 Colab에서 제공하는 VM은, 이그나이트가 사전 설치되어 있지 않은 상태이다. 따라서 다음과 같이 이그나이트의 최신 version을 설치한다. 참고로 pip 명령어는 package installer for python)의 약자이며, 아래 명령문 실행 시 이그나이트의 pre-release를 PyPI(python package index)로부터 설치하게 된다.


In [1]:
!pip install --pre pytorch-ignite

     |████████████████████████████████| 233 kB 8.2 MB/s 


결과에 따르면, 본 가이드라인 작성 시점의 최신 버전인 xxxx년 x월 xx일자 (예: 2021년 9월 10일자) 이그나이트 x.x.x (예: 0.5.0)가 설치 되었음을 알 수 있다..

#### 2.1.1.2. 노드 1개, 노드 당 프로세스 수 2개 실행

 설치가 완료되었다면, 아래의 코드를 실행한다. 코드에서 수행하는 작업은 아래와 같다.

 (라인 1)에서는 이그나이트의 distributed 패키지를 idist라는 이름으로 불러들이고, (라인 26-28)에서는 백엔드로 gloo를 할당하는 등의 설정 작업을 지정한 후, (라인 22)에서의 training 함수를, (라인 30-31)에서 컨택스트 매니징이 가능한 idist.Parallel을 이용하여 run 시킨다.

 (라인 22)의 training 함수는 몇 가지 정보를 출력한 후, 무의미한 작업을 반복하도록 작성되었다.

 그리고 (라인 27)에서는 분산처리 설정에 해당하는 dist_configs 딕셔너리에 nproc_per_node 키에 해당하는 값으로 2를 설정하였다. 이는 2개의 자식 프로세스를 생성(spawn)하여 분산 처리를 진행하라는 명령으로 생각하면 된다.


In [2]:
import ignite.distributed as idist

from functools import wraps
import time
import random

def fn_timer(function):
    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.time()
        result = function(*args, **kwargs)
        t1 = time.time()
        print (idist.get_rank(), " : Total time running %s: %s seconds" 
               % (function.__name__, str(t1-t0)))
        return result
    return function_timer

@fn_timer
def random_sort(n):
    return sorted([random.random() for i in range(n)])

def training(local_rank, config, **kwargs):
    print(idist.get_rank(), ': run with config:', config, '- backend=', idist.backend())
    random_sort(2500000)

backend = 'gloo' # or "xla-tpu" or None
dist_configs = {'nproc_per_node': 1, "start_method": "fork"}  # or dist_configs = {...}
config = {'c': 12345}

with idist.Parallel(backend=backend, **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=2)

2021-09-10 23:43:53,522 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'gloo'
2021-09-10 23:43:53,524 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 1
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-09-10 23:43:53,526 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f3988122950>' in 1 processes


0 : run with config: {'c': 12345} - backend= gloo
0  : Total time running random_sort: 1.3806157112121582 seconds


2021-09-10 23:43:55,149 ignite.distributed.launcher.Parallel INFO: End of run


결과의 첫번 째 출력 항목은 아래와 같으며, distributed launcher가 gloo 백엔드로 초기화 되었음을 표시한다.
  
*2021-06-25 07:48:33,646 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'gloo'*
<br/>
<br/>  

 상기 ‘’’1.4 분산 딥러닝 기본 지식’’’ 항목에서 언급된 바와 같이 분산처리를 위해서는 컴퓨팅 코어 간 통신이 필요하며, 본 예제에서는 컴퓨팅 코어가 CPU인 경우이므로 gloo나 mpi이 백엔드로 이용된 것이다. 그리고 만일 이 컴퓨팅 코어가 GPU인 경우 nccl 백엔드 이용이 가능하다. 

 이와 관련해서는 DISTRIBUTED COMMUNICATION PACKAGE - TORCH.DISTRIBUTED 페이지의 [rule of thumb](https://pytorch.org/docs/stable/distributed.html) 항목을 참조한다. ~~분산처리의 역사만큼이나 다양한 분산처리 방식이 존재한다. TCP 등을 이용해 직접 프로세스간 통신을 처리하는 방법도 있지만, 하이레벨 관점에서 CPU는 gloo, GPU는 nccl, TPU는 xla 백엔드를 사용해야 한다고 생각하는 것이 정신건강에 좋다.~~


In [ ]:
dist_configs['nproc_per_node'] = 2

with idist.Parallel(backend=backend, **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=2)

2021-06-28 09:34:53,152 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'gloo'
2021-06-28 09:34:53,154 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 2
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-06-28 09:34:53,156 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7fb2e079db00>' in 2 processes


0 : run with config: {'c': 12345} - backend= gloo
1 : run with config: {'c': 12345} - backend= gloo
1  : Total time running random_sort: 2.368380308151245 seconds
0  : Total time running random_sort: 2.4123685359954834 seconds


2021-06-28 09:34:55,941 ignite.distributed.launcher.Parallel INFO: End of run


In [ ]:
dist_configs['nproc_per_node'] = 8

with idist.Parallel(backend=backend, **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=2)

2021-06-28 09:34:55,957 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'gloo'
2021-06-28 09:34:55,963 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 8
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-06-28 09:34:55,973 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7fb2e079db00>' in 8 processes


3 : run with config: {'c': 12345} - backend= gloo
2 : run with config: {'c': 12345} - backend= gloo
5 : run with config: {'c': 12345} - backend= gloo
4 : run with config: {'c': 12345} - backend= gloo
7 : run with config: {'c': 12345} - backend= gloo
1 : run with config: {'c': 12345} - backend= gloo
0 : run with config: {'c': 12345} - backend= gloo
6 : run with config: {'c': 12345} - backend= gloo
1  : Total time running random_sort: 10.030449867248535 seconds
7  : Total time running random_sort: 10.167544603347778 seconds
0  : Total time running random_sort: 10.17899227142334 seconds
4  : Total time running random_sort: 10.21057939529419 seconds
3  : Total time running random_sort: 10.241017818450928 seconds
5  : Total time running random_sort: 10.342675685882568 seconds
2  : Total time running random_sort: 10.453433752059937 seconds
6  : Total time running random_sort: 10.57030439376831 seconds


2021-06-28 09:35:08,860 ignite.distributed.launcher.Parallel INFO: End of run


In [ ]:
dist_configs['nproc_per_node'] = 50

with idist.Parallel(backend=backend, **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=2)

2021-06-28 09:35:08,874 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'gloo'
2021-06-28 09:35:08,886 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 50
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-06-28 09:35:08,889 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7fb2e079db00>' in 50 processes


42 : run with config: {'c': 12345} - backend= gloo
36 : run with config: {'c': 12345} - backend= gloo
34 : run with config: {'c': 12345} - backend= gloo
37 : run with config: {'c': 12345} - backend= gloo
32 : run with config: {'c': 12345} - backend= gloo
18 : run with config: {'c': 12345} - backend= gloo
15 : run with config: {'c': 12345} - backend= gloo
33 : run with config: {'c': 12345} - backend= gloo
24 : run with config: {'c': 12345} - backend= gloo
31 : run with config: {'c': 12345} - backend= gloo
38 : run with config: {'c': 12345} - backend= gloo
23 : run with config: {'c': 12345} - backend= gloo
41 : run with config: {'c': 12345} - backend= gloo
25 : run with config: {'c': 12345} - backend= gloo
20 : run with config: {'c': 12345} - backend= gloo
44 : run with config: {'c': 12345} - backend= gloo
27 : run with config: {'c': 12345} - backend= gloo
21 : run with config: {'c': 12345} - backend= gloo
22 : run with config: {'c': 12345} - backend= gloo
28 : run with config: {'c': 123

2021-06-28 09:36:24,578 ignite.distributed.launcher.Parallel INFO: End of run


In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: